# MICROCHIP SINGLE ELECTRON RATE CALCULATION

In [1]:
from functions_py import *
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
plt.rcParams.update({
    "image.origin": "lower",
    "image.aspect": 1,
    #"text.usetex": True,
    "grid.alpha": .5,
    "axes.linewidth":2,
    "lines.linewidth" : 1,
    "font.size":    15.0,
    "xaxis.labellocation": 'right',  # alignment of the xaxis label: {left, right, center}
    "yaxis.labellocation": 'top',  # alignment of the yaxis label: {bottom, top, center}
    "xtick.top":           True ,  # draw ticks on the top side
    "xtick.major.size":    8    ,# major tick size in points
    "xtick.minor.size":    4      ,# minor tick size in points
    "xtick.direction":     'in',
    "xtick.minor.visible": True,
    "ytick.right":           True ,  # draw ticks on the top side
    "ytick.major.size":    8    ,# major tick size in points
    "ytick.minor.size":    4      ,# minor tick size in points
    "ytick.direction":     'in',
    "ytick.minor.visible": True,
    "ytick.major.width":   2   , # major tick width in points
    "ytick.minor.width":   1 ,
    "xtick.major.width":   2   , # major tick width in points
    "xtick.minor.width":   1 ,
    "legend.framealpha": 0 ,
    "legend.loc": 'best',

})

In [3]:
import numpy.ma as ma

In [4]:

def LocalSER(data,mask,sigma,readout_time,extensions=4):
    
    madata=ma.masked_array(data, mask)
    if extensions!=1:
        
        dc_list=[]
        dc_err_list=[]
        for k in range(extensions):
            global_mask=mask[k]
            Npix=global_mask.shape[0]*global_mask.shape[1]

            time_map=np.linspace(0,readout_time,num=Npix,endpoint=True).reshape(global_mask.shape[0],global_mask.shape[1])

            time_map_mask=ma.masked_where( global_mask.astype(bool), time_map)
            Nactive=ma.count(time_map_mask)
            exposure_time=ma.mean(time_map_mask)
#             print(Nactive,exposure_time)
            sigmaN=sigma[k]
            
            def poisson_normN(x, mu, A, lamb, Nmax=5): #sigma parameter global 
                y = 0.
                for i in range(0, Nmax+1):
                    y += (lamb**i)/float(math.factorial(i)) *np.exp(-0.5*((x-i-mu-lamb)/float(sigmaN))**2)
                return A*np.exp(-lamb)*y/(np.sqrt(2*np.pi*sigmaN**2))
            
            try:
                masked_hist, bins = np.histogram( ma.compressed(madata[k][convolution_mask]), np.arange(-0.5, 2.5, .01) )
                x = (bins[1:]+bins[:-1])/2
                
                popt, pcov = curve_fit( 
                    poisson_normN, 
                    x, 
                    masked_hist, 
                    p0=[-0.4, 1000, 0.05],
                )
                perr = np.sqrt(np.diag(pcov))
                dc = popt[2]/exposure_time # electron/pix/day
                dc_err = perr[2]/exposure_time # electron/pix/day                
            except (RuntimeError,OptimizeWarning,RuntimeWarning):
                print( f"Error - dc fit failed at" )
                (dc, dc_err)=(-1000,-1000)
            dc_list.append(dc)
            dc_err_list.append(dc_err)
        return (dc_list,dc_err_list)

In [5]:
path='/share/storage2/connie/data/microchip/proc_bkgd_1x2_2/proc_skp_moduleC002_bkgd_clearVB300_NBINROW2_img75.fits'


hdu_list = fits.open(path)
readout_time = GetRDtime(hdu_list)
data_pre = precal(hdu_list,extensions=4)
gain, gain_err, data= LocalCalib(data_pre,extensions=4)
sigma, sigma_err = LocalSigma(data,extensions=4)


mask_bleeding_array, HE_events_array = mask_bleeding(data,direction='xy',iterations=[40,20],extensions=4,he_th=80)
SRE_mask_array=mask_SRE(DataMminus(data,(mask_bleeding_array+HE_events_array)),extensions=4,SRE_th=1.5)

masks_data=[32*mask_bleeding_array.astype('bool'),128*SRE_mask_array.astype('bool')]
GlobalMask=np.array(masks_data).max(axis=0)
EventHalo_Mask(data,mask_bleeding_array)

event_halo_SR_mask = np.array(GlobalMask,dtype=bool)|EventHalo_Mask(data,mask_bleeding_array).astype(bool)

In [7]:
ser_list,ser_err_list = LocalSER(data,event_halo_SR_mask,sigma,readout_time)

In [10]:
ser_list

[9.928524842520893, 2.0244499800277183, 7.615579253856292, 26.00399941857452]

In [14]:
np.load('DataADU_1x10_1.npy').shape

(854, 4, 33995)